## Crawling and Extracting Data from Websites

This module uses a set of non-standard libraries, which need to be installed on your machine. By default, your instance should have these installed, but if this is not the case, type these in the Unix shell prompt





In [ ]:
!sudo -H apt-get -y install libxml2-dev libxslt-dev python3-dev

and then

In [ ]:
!sudo -H pip3 install -U lxml

In [ ]:
!sudo -H pip3 install -U pandas

### Fetching the headlines from ESPN.com

Let's start by trying to fetch the headlines from the site ESPN.com.



In [ ]:
import requests # This command allows us to fetch URLs
from lxml import html # This module will allow us to parse the returned HTML/XML
import pandas # To create a dataframe

# Let's start by fetching the page, and parsing it
url = "http://www.espn.com/"
response = requests.get(url) # get the html of that url
doc = html.fromstring(response.text) # parse it and create a document

The `doc` variable is an `HtmlElement` object, and we can now use **XPath** queries to locate the elements that we need. (Depending on time, we may do in class a tutorial on XPath. For now, you can look at the [W3Schools tutorial](http://www.w3schools.com/xpath/xpath_nodes.asp))

For example, to find all the `<a ...> ... </a>` tags in the returned html, which store the links in the page, we issue the command:

In [ ]:
links = doc.xpath("//a")
len(links)

In [ ]:
lnk = links[70]
type(lnk)


The `lnk` variable is again an HtmlElement. To get parts of the html element that we need, we can use the `get` method (e.g., to get the `href` attribute) and the `text_content()` method (to get the text within the `<a>...</a>` tag.

In [ ]:
lnk.get("href")

In [ ]:
lnk.text_content()

Now, let's revisit the _list comprehension_ approach that we discussed in the Python Primer session, for quickly constructing lists:

In [ ]:
urls = [lnk.get("href") for lnk in doc.xpath('//a')]
urls

#### Exercise

Use a list compresension approach, to get the text_content of all the URLs in the page.

In [ ]:
# your code here

And now create a list where we put together text content and the URL for each link

In [ ]:
# your code here

#### Let's get the headlines...

Now, let's examine how we can get the data from the website. The key is to understand the structure of the HTML, where the data that we need is stored, and how to fetch the elements. Then, using the appropriate XPath queries, we will get what we want.

In [ ]:
import requests # This command allows us to fetch URLs
from lxml import html # This module will allow us to parse the returned HTML/XML
import pandas # To create a dataframe

Let's start by fetching the page, and parsing it

In [ ]:
url = "http://www.espn.com/"
response = requests.get(url) # get the html of that url
doc = html.fromstring(response.text) # parse it and create a document

By using the `"Right-Click > Inspect"` option of Chrome,
we right click on the headlines and select `"Inspect"`.
This opens the source code.
There we see that all under a `<div class="headlineStack">` tag
which is also the only such tag in the html source

In [ ]:
headlineNode = doc.xpath('//div[@class="headlineStack"]')[0]

Each headline is under a  `<li><a href="...."></a>` tag.
So, we get all the `<li><a ...>` tags within the `<div class="headlineStack">`
(which is stored in the "`headlineNode`" variable)

In [ ]:
headlines = headlineNode.xpath('.//ul/li/a')

Now, we have the nodes with the conent in the headlines variable.
We extract the text and the URL.

In [ ]:
data = [{"Title": a.text_content(), "URL": a.get("href")} for a in headlines]
data

And let's create our dataframe, so that we can have a better view

In [ ]:
dataframe = pandas.DataFrame(data)
dataframe

### In Class Example: Crawl BuzzFeed

* We will try to get the top articles that appear on Buzzfeed
* We will grab the link for the article, the text of the title, the description, and the editor.
* The results will be stored in a dataframe (we will see in detail what a dataframe is, in a couple of modules)


In [ ]:
#your code here
import requests
from lxml import html

resp = requests.get("http://www.buzzfeed.com")
doc = html.fromstring(resp.text)



#### Solution for Buzzfeed (as of February 27, 2017)

In [ ]:
import requests # This command allows us to fetch URLs
from lxml import html # This module will allow us to parse the returned HTML/XML
import pandas

# Let's start by fetching the page, and parsing it
url = "http://www.buzzfeed.com/"
response = requests.get(url) # get the html of that url
doc = html.fromstring(response.text) # parse it and create a document

articleNodes = doc.xpath("//div[@data-module='card-article']") 

def parseArticleNode(article):
    headline = article.xpath(".//a/h2")[0].text_content()
    headline_link = article.xpath(".//a/h2")[0].text_content()
    description = article.xpath(".//a/p")[0].text_content()
    editor = article.xpath(".//a[contains(@class,'card__byline-link')]/span")[0].text_content().strip()
    
    result = {
        "headline": headline,
        "URL" : headline_link,
        "description" : description,
        "editor" : editor
    }
    return result

data = [parseArticleNode(article) for article in articleNodes]
df = pandas.DataFrame(data)
df